In [1]:
import os
import random
import pandas as pd

path = "./Kickstarter_Dataset/"

dataframes = [pd.read_csv(filename) for filename in os.scandir(path) if filename.is_file]

result = pd.concat(dataframes).reset_index()

result.to_csv("./data/kickstarter_combined.csv")

In [2]:
import json
import random
from tqdm import tqdm


def find_dict_strings(df, sample_size=5):
    """
    find string dictionary columns in a dataframe

    :param df:
    :param sample_size:
    :return:
    """
    sdicts = [0]*len(df.columns)
    sample = [random.randint(0, df.shape[0]) for _ in range(sample_size)]
    df_sample = df.filter(items=sample, axis=0).reset_index()
    for index, row in df_sample.iterrows():
        for idx, col in enumerate(df.columns):
            if type(df_sample.iat[index, idx]) == str and df_sample.iat[index, idx][0] == '{':
                sdicts[idx] = sdicts[idx] + 1
    return [df.columns[i-1] for i, item in enumerate(sdicts) if item == sample_size]

def strdict_to_dict(x):
    """
    convert string dictionary into dictionary

    :param x:
    :param errors:
    :return:
    """
    try:
        return json.loads(x)
    except:
        return x

def apply_strdict_to_dict(df):
    """

    :param df:
    :return:
    """
    for i in sd:
        df[i] = df[i].apply(lambda x: strdict_to_dict(x))
    return df

def calculate_errors(df):
    """
    calcaute the overall errors trying to convert str dict to dict

    :param df:
    :return:
    """
    errors = []
    with tqdm(total=df.shape[0]*len(sd)) as pbar:
        for col in sd:
            for index, row in df.iterrows():
                if type(df.iloc[index][col]) != dict:
                    errors.append(index)
                pbar.update(1)

    print(f"Number of errors detected: {len(errors)}")
    return errors

def split_dict(df_clean):
    """
    split dictionaries into columns

    :param df_clean:
    :return:
    """
    dfs = []
    for dict_col in sd:
        a = pd.DataFrame(df_clean[dict_col].tolist())
        col_names = {i: f"{i}_{dict_col}" for i in a.columns}
        a.rename(columns=col_names, inplace=True)
        dfs.append(a)
    return dfs


# Find string dicts and calculate errors
sd = find_dict_strings(result)
result = apply_strdict_to_dict(result)
errors = calculate_errors(result)

# Filter dataframe by errors
df = result.filter(items=errors, axis=0)
df.to_csv("./data/errors.csv")

# Filter dataframe by clean data
df_clean = result[~result.index.isin(errors)]
df_clean.to_csv("./data/clean_data.csv")

# Split dicts into columns for clean dataframe
dfs = split_dict(df_clean)
horizontal_concat = pd.concat(dfs, axis=1)

# combine split dataframe and clean data and drop old columns
df_clean_split = pd.concat([df_clean, horizontal_concat], axis=1)
df_clean_split.drop(columns=sd, inplace=True)
df_clean_split.to_csv("./data/df_clean_split.csv")

100%|██████████| 1234176/1234176 [03:39<00:00, 5621.03it/s]


Number of errors detected: 4319
